# Metadata

**L1 Taxonomy** - Backend Integration

**L2 Taxonomy** - API Integration

**Subtopic** - Using GraphQL introspection from Python to adapt to API schema changes

**Use Case** - Develop a Python module that uses GraphQL introspection to adapt to API schema changes. The module should be able to retrieve the current schema from a GraphQL API, parse it, and adjust its own internal data structures to match. It should also be able to handle changes in the schema over time, such as new fields being added or existing fields being removed.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
```


# Prompt
Problem Statement:
- Develop a Python module that uses GraphQL introspection to adapt to API schema changes.
- The module should be able to retrieve the current schema from a GraphQL API, parse it, and adjust its own internal data structures to match.
- It should also be able to handle changes in the schema over time, such as new fields being added or existing fields being removed.
- Please make sure not to include unnecessary explanations or extra details. Just provide the complete implementation from start to end as per the prompt.

Input Format:
- A string representing the GraphQL API endpoint (e.g., "https://example.com/graphql").
- A dictionary of optional HTTP headers (e.g., {"Authorization": "Bearer token", "Content-Type": "application/json"}).
- An optional previous schema snapshot as a dictionary to compare against the current schema.
- An optional dictionary of configuration flags to control behavior (e.g., {"validate_changes": True, "print_diff": False}).

Input Constraints:
- The GraphQL API endpoint must be a valid HTTP or HTTPS URL.
- Headers, if provided, must be in dictionary format with string keys and values.
- The schema snapshot must be either:
  - A valid JSON object conforming to GraphQL introspection format.
- Configuration flags must be a dictionary containing only recognized keys:
  - "validate_changes": boolean
  - "print_diff": boolean
  - "update_internal_state": boolean
- The API must support the GraphQL introspection query (`__schema` field must be available).
- Network requests must complete within a reasonable timeout (e.g., 10 seconds).

Output Format:
- A dictionary representing the current GraphQL schema as returned by the introspection query ({"__schema": ...}).
- If a previous schema is provided, an additional dictionary indicating the differences:
  - "added_types": list of newly added types or fields
  - "removed_types": list of removed types or fields
  - "changed_fields": list of fields whose types, arguments, or nullability have been modified.
   - each item in the list is a dict.
- If no previous schema is provided, only the current schema will be returned and comparison fields will be omitted.


Class Definition:
- class GraphQLSchemaAdapter:
    - __init__(self, endpoint: str, headers: dict = None, config: dict = None)
        - Initializes the adapter with the given GraphQL endpoint, optional headers, and configuration flags.

    - fetch_current_schema(self) -> dict
        - Sends an introspection query to the GraphQL endpoint and returns the current schema as a dictionary.

    - load_previous_schema(self, source: dict) -> dict
        - Loads a previous schema directly from a dictionary.

    - compare_schemas(self, old_schema: dict, new_schema: dict) -> dict
        - Compares two schemas and returns the differences as a dictionary with keys like 'added_types', 'removed_types', etc.

    - update_internal_state(self, schema_diff: dict) -> None
        - Updates internal representations based on schema changes, if enabled via config.

    - run(self, old_schema: dict = None) -> dict
        - Orchestrates the process: fetches current schema, compares with previous (if provided), updates state, and returns results.

# Requirements
Explicit Requirements:
- The module must connect to a GraphQL endpoint and perform an introspection query.
- It must parse and return the current schema in dictionary format.
- The module must identify added, removed, or changed fields/types.
- It must support headers (e.g., Authorization) for secured GraphQL endpoints.
- Custom exceptions like SchemaFormatError, NetworkError, and ValidationError must be used for clear error reporting.

Implicit Requirements:
- The module should handle network errors gracefully (e.g., timeouts, connection errors).
- It should validate the structure of the GraphQL introspection response.
- It should support reusability and modular design (e.g., well-structured methods and classes).
- It should avoid unnecessary reprocessing if the schema has not changed.
- It should maintain backward compatibility with older schema versions when possible.
- It should be extendable for future schema evolution strategies.
- All data structures and comparisons must be deterministic and consistent.

Solution Expectations:
- A well-structured Python module with clear class and method definitions.
- The class should encapsulate all functionality related to schema retrieval, comparison, and adaptation.
- The module must correctly perform a GraphQL introspection query and parse the result.
- It should detect and report schema changes such as added, removed, or modified types and fields.
- The code should include input validation, error handling, and meaningful exceptions.
- Internal data structures should be automatically updated if configured.
- The design should be modular, reusable, and easily testable.
- The solution should be able to handle evolving GraphQL APIs without manual intervention.

Edge Cases and Behaviour:
- Empty or Invalid Endpoint:
  - The module should raise a clear error if the endpoint is missing or not a valid URL.

- Unreachable GraphQL Server:
  - Should handle connection errors, timeouts, or DNS failures gracefully with retry or failure messages.

- Introspection Not Supported:
  - If the GraphQL server disables introspection, the module should report this with a meaningful exception.

- Missing or Malformed Previous Schema:
  - If a provided schema is an invalid JSON, the module should fall back gracefully or raise a descriptive error.

- No Schema Changes:
  - If the new schema is identical to the previous one, the module should report "no changes detected" and skip updates.

- Addition of New Fields or Types:
  - Should list these in the `added_types` or equivalent output key.

- Removal of Fields or Types:
  - Should list these in the `removed_types` or equivalent key, and update or warn depending on configuration.

- Type Modifications (e.g., field type changed):
  - Should capture these in `changed_fields` and ensure consistency in internal updates.

- Large or Deeply Nested Schemas:
  - Should efficiently parse and compare without exceeding memory or recursion limits.

- Authenticated Endpoints:
  - Should correctly apply headers (e.g., tokens) and report unauthorized access errors.

- Corrupted Introspection Response:
  - Should validate the structure before processing and raise a schema format error if needed.

- Valid HTTP response with invalid or incomplete introspection data:
  - Should detect missing "__schema" or unexpected types and raise a SchemaFormatError.

Solution Constraints:
- Must use only standard Python libraries or widely accepted third-party packages (e.g., requests, json).
- Must support GraphQL introspection query as per the GraphQL specification.
- The solution should not depend on specific API implementations or hardcoded schema structures.
- Must handle schema comparison deterministically and consistently.
- Response parsing and diff computation must complete within reasonable time (e.g., <5 seconds for medium schemas).
- Must support schemas of realistic size (e.g., thousands of types and fields) without excessive memory usage.
- Internal state updates must not mutate input schema objects directly.
- All external network calls must respect timeout settings (recommended default: 10 seconds).
- Implement local in-memory caching of fetched schemas to minimize repeated network requests.
- The module's peak memory usage must not exceed 100 MB during schema fetching and comparison, even for very large schemas.

In [ ]:
# code
"""
GraphQL Schema Adapter.

This module defines a Python class to adapt to GraphQL schema changes.

It uses introspection API and supports schema comparison,
error handling, caching, and memory safety.
"""

import requests
import re
import threading
import copy
import resource


class SchemaFormatError(Exception):
    """Raised when the schema is malformed or missing required keys."""

    pass


class NetworkError(Exception):
    """Raised when there is a network communication issue."""

    pass


class ValidationError(Exception):
    """Raised when validation of a user argument fails."""

    pass


_INTROSPECTION_QUERY = """
query IntrospectionQuery {
  __schema {
    queryType { name }
    mutationType { name }
    subscriptionType { name }
    types {
      ...FullType
    }
    directives {
      name
      description
      locations
      args {
        ...InputValue
      }
    }
  }
}

fragment FullType on __Type {
  kind
  name
  description
  fields(includeDeprecated: true) {
    name
    description
    args {
      ...InputValue
    }
    type {
      ...TypeRef
    }
    isDeprecated
    deprecationReason
  }
  inputFields {
    ...InputValue
  }
  interfaces {
    ...TypeRef
  }
  enumValues(includeDeprecated: true) {
    name
    description
    isDeprecated
    deprecationReason
  }
  possibleTypes {
    ...TypeRef
  }
}

fragment InputValue on __InputValue {
  name
  description
  type { ...TypeRef }
  defaultValue
}

fragment TypeRef on __Type {
  kind
  name
  ofType {
    kind
    name
    ofType {
      kind
      name
      ofType {
        kind
        name
        ofType {
          kind
          name
        }
      }
    }
  }
}
"""


def _is_valid_url(url):
    """Check if the provided URL is valid for HTTP/HTTPS."""
    return bool(re.match(r'^https?://[^\s/$.?#].[^\s]*$', url))


def _build_type_signature(type_obj):
    """Recursively construct a type string from introspection type."""
    if not type_obj:
        return "None"
    kind = type_obj.get("kind")
    name = type_obj.get("name")
    of_type = type_obj.get("ofType")
    if kind == "NON_NULL":
        return f"{_build_type_signature(of_type)}!"
    if kind == "LIST":
        return f"[{_build_type_signature(of_type)}]"
    return name or (_build_type_signature(of_type) if of_type else "Unknown")


def _extract_types(schema):
    """Extract a name->type mapping of types from a schema object."""
    return {
        t.get("name"): t
        for t in schema.get("__schema", {}).get("types", [])
        if t.get("name")
    }


def _extract_fields(type_obj):
    """Extract a mapping of field names to type/args for one GraphQL type."""
    result = {}
    for field in (type_obj.get("fields", []) or []):
        args = {
            a.get("name"): {
                "type": _build_type_signature(a.get("type")),
                "default": a.get("defaultValue")
            }
            for a in (field.get("args", []) or [])
        }
        result[field.get("name")] = {
            "type": _build_type_signature(field.get("type")),
            "args": args
        }
    return result


def _get_memory_usage_mb():
    """Calculate memory usage in MB."""
    try:
        usage = resource.getrusage(resource.RUSAGE_SELF)
        return usage.ru_maxrss / 1024 if hasattr(usage, 'ru_maxrss') else 0
    except Exception:
        return 0


class GraphQLSchemaAdapter:
    """Adapter for dynamic GraphQL schema introspection, change detection."""

    _schema_cache = {}
    _cache_lock = threading.Lock()

    def __init__(self, endpoint, headers=None, config=None):
        """Initialize the adapter with an endpoint, headers, and configuration.

        Args:
            endpoint (str): GraphQL endpoint URL (must be valid HTTP/HTTPS).
            headers (dict): Optional HTTP headers.
            config (dict): Optional config flags.
        Raises:
            ValidationError: If the endpoint is not valid.
        """
        if not (isinstance(endpoint, str) and _is_valid_url(endpoint)):
            raise ValidationError("Invalid GraphQL endpoint URL.")
        self.endpoint = endpoint
        self.headers = headers if headers and isinstance(headers, dict) else {}
        self.config = {
            "validate_changes": True,
            "print_diff": False,
            "update_internal_state": True,
        }
        if config:
            for key in config:
                if key in self.config:
                    self.config[key] = config[key]
                else:
                    raise ValidationError(f"Unsupported config key: {key}")
        self.internal_state = None

    def fetch_current_schema(self):
        """Perform an introspection query and return the current schema.

        Returns:
            dict: Introspection schema object as returned from endpoint.

        Raises:
            NetworkError, SchemaFormatError, MemoryError
        """
        cache_key = (self.endpoint, frozenset(self.headers.items()))
        with GraphQLSchemaAdapter._cache_lock:
            cached = GraphQLSchemaAdapter._schema_cache.get(cache_key)
            if cached:
                return copy.deepcopy(cached)
        try:
            resp = requests.post(
                self.endpoint,
                json={"query": _INTROSPECTION_QUERY},
                headers=self.headers,
                timeout=10,
            )
            if resp.status_code == 401:
                raise NetworkError("Unauthorized access"
                                   " to the endpoint (401).")
            resp.raise_for_status()
        except requests.exceptions.RequestException as exc:
            raise NetworkError(f"Failed to fetch schema from endpoint: {exc}")

        try:
            data = resp.json()
        except Exception as exc:
            raise SchemaFormatError(
                f"Failed to parse JSON from schema response: {exc}"
            )

        if (
            not isinstance(data, dict)
            or "data" not in data
            or "__schema" not in data["data"]
        ):
            raise SchemaFormatError("Introspection result "
                                    "missing '__schema' key.")
        schema = {"__schema": data["data"]["__schema"]}

        with GraphQLSchemaAdapter._cache_lock:
            GraphQLSchemaAdapter._schema_cache[
                cache_key] = copy.deepcopy(schema)

        if _get_memory_usage_mb() > 100:
            raise MemoryError(
                "Memory usage exceeded 100 MB limit during schema fetch."
            )

        return schema

    def load_previous_schema(self, source):
        """Validate and return the provided previous schema dict."""
        if not source:
            return None
        if not (isinstance(source, dict) and "__schema" in source):
            raise SchemaFormatError(
                "Provided previous schema must"
                " be a dict with a '__schema' key."
            )
        return source

    def compare_schemas(self, old_schema, new_schema):
        """Compare old and new schemas and return a diff.

        Args:
            old_schema (dict): Previous schema object.
            new_schema (dict): Latest schema object.

        Returns:
            dict: Diff dict with 'added_types',
            'removed_types', and 'changed_fields'.
        """
        old_types = _extract_types(old_schema) if old_schema else {}
        new_types = _extract_types(new_schema)

        added_types = []
        removed_types = []
        changed_fields = []

        for t_name, t in new_types.items():
            if t_name not in old_types:
                added_types.append({"type": t_name, "kind": t.get("kind")})

        for t_name, t in old_types.items():
            if t_name not in new_types:
                removed_types.append({"type": t_name, "kind": t.get("kind")})

        for t_name in new_types:
            if t_name not in old_types:
                continue
            old_fields = _extract_fields(old_types[t_name])
            new_fields = _extract_fields(new_types[t_name])

            for f_name, f_data in new_fields.items():
                if f_name not in old_fields:
                    added_types.append(
                        {"type": t_name, "field": f_name, "action": "added"}
                    )
            for f_name in old_fields:
                if f_name not in new_fields:
                    removed_types.append(
                        {"type": t_name, "field": f_name, "action": "removed"}
                    )
            for f_name in old_fields:
                if f_name in new_fields:
                    old_f = old_fields[f_name]
                    new_f = new_fields[f_name]
                    if (old_f["type"] != new_f["type"]) or (
                        old_f["args"] != new_f["args"]
                    ):
                        changed_fields.append(
                            {
                                "type": t_name,
                                "field": f_name,
                                "old_type": old_f["type"],
                                "new_type": new_f["type"],
                                "old_args": old_f["args"],
                                "new_args": new_f["args"],
                            }
                        )

        diff = {
            "added_types": added_types,
            "removed_types": removed_types,
            "changed_fields": changed_fields,
        }

        if _get_memory_usage_mb() > 100:
            raise MemoryError(
                "Memory usage exceeded 100 MB during schema comparison."
            )

        return diff

    def update_internal_state(self, schema_diff):
        """Update the object's state with schema_diff, if configured."""
        if self.config.get("update_internal_state", True):
            self.internal_state = copy.deepcopy(schema_diff)

    def run(self, old_schema=None):
        """Orchestrate schema fetch, diff, state update, and return results.

        Args:
            old_schema (dict): Previous schema, or None.

        Returns:
            dict: Result, including 'current_schema' and, if present, the diff.
        """
        current_schema = self.fetch_current_schema()
        result = {"current_schema": copy.deepcopy(current_schema)}
        if old_schema:
            old_loaded = self.load_previous_schema(old_schema)
            diff = self.compare_schemas(old_loaded, current_schema)
            result.update(diff)
            if self.config.get("update_internal_state", True):
                self.update_internal_state(diff)
        return result


In [ ]:
# tests

"""Unit tests for GraphQLSchemaAdapter class in main module."""

import unittest

from main import (
    GraphQLSchemaAdapter,
    SchemaFormatError,
    NetworkError,
    ValidationError
)


class TestGraphQLSchemaAdapter(unittest.TestCase):
    """Test GraphQLSchemaAdapter behavior for all key functionalities."""

    def setUp(self):
        """Initialize common schema, endpoint, headers, and config."""
        self.valid_endpoint = "https://example.com/graphql"
        self.invalid_endpoint = "ftp://invalid-url"
        self.headers = {"Authorization": "Bearer token"}
        self.config = {
            "validate_changes": True,
            "print_diff": True,
            "update_internal_state": True
        }
        self.minimal_schema = {
            "__schema": {
                "types": [
                    {
                        "name": "Query",
                        "kind": "OBJECT",
                        "fields": [
                            {
                                "name": "hello",
                                "type": {"kind": "SCALAR", "name": "String"},
                                "args": []
                            }
                        ]
                    }
                ]
            }
        }
        self.modified_schema = {
            "__schema": {
                "types": [
                    {
                        "name": "Query",
                        "kind": "OBJECT",
                        "fields": [
                            {
                                "name": "hello",
                                "type": {"kind": "SCALAR", "name": "Int"},
                                "args": []
                            },
                            {
                                "name": "newField",
                                "type": {"kind": "SCALAR", "name": "String"},
                                "args": []
                            }
                        ]
                    },
                    {
                        "name": "ExtraType",
                        "kind": "OBJECT",
                        "fields": []
                    }
                ]
            }
        }

    def test_invalid_endpoint_raises_validation_error(self):
        """Raise ValidationError for invalid endpoint."""
        with self.assertRaises(ValidationError):
            GraphQLSchemaAdapter(self.invalid_endpoint)

    def test_unsupported_config_key_raises_validation_error(self):
        """Raise ValidationError for unknown config key."""
        with self.assertRaises(ValidationError):
            GraphQLSchemaAdapter(
                self.valid_endpoint,
                config={"bad_key": True}
            )

    def test_load_previous_schema_with_invalid_dict_raises_error(self):
        """Raise SchemaFormatError for invalid previous schema."""
        adapter = GraphQLSchemaAdapter(self.valid_endpoint)
        with self.assertRaises(SchemaFormatError):
            adapter.load_previous_schema({"not_schema": {}})

    def test_load_previous_schema_with_none_returns_none(self):
        """Return None when previous schema is None."""
        adapter = GraphQLSchemaAdapter(self.valid_endpoint)
        self.assertIsNone(adapter.load_previous_schema(None))

    def test_compare_schemas_detects_added_and_removed_types(self):
        """Detect added and removed types between schemas."""
        adapter = GraphQLSchemaAdapter(self.valid_endpoint)
        diff = adapter.compare_schemas(
            self.minimal_schema,
            self.modified_schema
        )
        added = [t["type"] for t in diff["added_types"]]
        removed = [t["type"] for t in diff["removed_types"]]
        self.assertIn("ExtraType", added)
        self.assertEqual(len(removed), 0)

    def test_compare_schemas_detects_added_and_removed_fields(self):
        """Detect field additions and removals in types."""
        adapter = GraphQLSchemaAdapter(self.valid_endpoint)
        mod_schema = {
            "__schema": {
                "types": [
                    {
                        "name": "Query",
                        "kind": "OBJECT",
                        "fields": [
                            {
                                "name": "hello",
                                "type": {"kind": "SCALAR", "name": "String"},
                                "args": []
                            }
                        ]
                    }
                ]
            }
        }
        diff = adapter.compare_schemas(self.modified_schema, mod_schema)
        removed = [t for t in diff["removed_types"] if "field" in t]
        self.assertTrue(any(f["field"] == "newField" for f in removed))

    def test_compare_schemas_detects_changed_fields(self):
        """Detect changes in field types."""
        adapter = GraphQLSchemaAdapter(self.valid_endpoint)
        diff = adapter.compare_schemas(
            self.minimal_schema,
            self.modified_schema
        )
        changed = [
            f for f in diff["changed_fields"] if f["field"] == "hello"
        ]
        self.assertTrue(changed)
        self.assertEqual(changed[0]["old_type"], "String")
        self.assertEqual(changed[0]["new_type"], "Int")

    def test_update_internal_state_applies_diff(self):
        """Update internal state using given diff."""
        adapter = GraphQLSchemaAdapter(self.valid_endpoint)
        diff = adapter.compare_schemas(
            self.minimal_schema,
            self.modified_schema
        )
        adapter.update_internal_state(diff)
        self.assertEqual(adapter.internal_state, diff)

    def test_run_returns_current_schema_only_if_no_old_schema(self):
        """Return only current schema when old schema is None."""
        adapter = GraphQLSchemaAdapter(self.valid_endpoint)
        adapter.fetch_current_schema = lambda: self.minimal_schema
        result = adapter.run()
        self.assertIn("current_schema", result)
        self.assertNotIn("added_types", result)

    def test_run_returns_diff_if_old_schema_provided(self):
        """Return schema diff when old schema is provided."""
        adapter = GraphQLSchemaAdapter(self.valid_endpoint)
        adapter.fetch_current_schema = lambda: self.modified_schema
        result = adapter.run(self.minimal_schema)
        self.assertIn("added_types", result)
        self.assertIn("removed_types", result)
        self.assertIn("changed_fields", result)

    def test_run_updates_internal_state_if_config_enabled(self):
        """Update internal state if config enables it."""
        adapter = GraphQLSchemaAdapter(
            self.valid_endpoint,
            config={"update_internal_state": True}
        )
        adapter.fetch_current_schema = lambda: self.modified_schema
        adapter.internal_state = None
        adapter.run(self.minimal_schema)
        self.assertIsNotNone(adapter.internal_state)

    def test_run_does_not_update_internal_state_if_config_disabled(self):
        """Do not update internal state if config disables it."""
        adapter = GraphQLSchemaAdapter(
            self.valid_endpoint,
            config={"update_internal_state": False}
        )
        adapter.fetch_current_schema = lambda: self.modified_schema
        adapter.internal_state = None
        adapter.run(self.minimal_schema)
        self.assertIsNone(adapter.internal_state)

    def test_schema_cache_returns_cached_schema(self):
        """Return cached schema for repeated same endpoint."""
        adapter = GraphQLSchemaAdapter(
            self.valid_endpoint,
            headers=self.headers
        )

        def fake_post(*args, **kwargs):
            class FakeResp:
                status_code = 200

                def json(self):
                    return {"data": {"__schema": {"types": []}}}

                def raise_for_status(self):
                    pass

            return FakeResp()

        import main
        main.requests.post = fake_post
        schema1 = adapter.fetch_current_schema()
        schema2 = adapter.fetch_current_schema()
        self.assertEqual(schema1, schema2)

    def test_fetch_current_schema_raises_network_error_on_401(self):
        """Raise NetworkError on unauthorized response."""
        adapter = GraphQLSchemaAdapter(self.valid_endpoint)

        def fake_post(*args, **kwargs):
            class FakeResp:
                status_code = 401

                def raise_for_status(self):
                    raise Exception("401 Unauthorized")

                def json(self):
                    return {"data": {"__schema": {"types": []}}}

            return FakeResp()

        import main
        main.requests.post = fake_post
        with self.assertRaises(NetworkError):
            adapter.fetch_current_schema()

    def test_schema_format_error_on_missing_schema(self):
        """Raise SchemaFormatError when '__schema' is missing."""
        adapter = GraphQLSchemaAdapter(self.valid_endpoint)

        def fake_post(*args, **kwargs):
            class FakeResp:
                status_code = 200

                def raise_for_status(self):
                    pass

                def json(self):
                    return {"data": {}}

            return FakeResp()

        import main
        main.requests.post = fake_post
        with self.assertRaises(SchemaFormatError):
            adapter.fetch_current_schema()


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/763cd8b4-69f3-4a96-9a6c-0de9eeeef66d?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F81794%2Fview

#### Model code:

```python
# code generated by the model
"""GraphQL Schema Adapter Module"""

import json
import requests
from urllib.parse import urlparse

INTROSPECTION_QUERY = """
query IntrospectionQuery {
  __schema {
    queryType { name }
    mutationType { name }
    subscriptionType { name }
    types {
      ...FullType
    }
    directives {
      name
      locations
      args {
        ...InputValue
      }
    }
  }
}

fragment FullType on __Type {
  kind
  name
  description
  fields(includeDeprecated: true) {
    name
    description
    args {
      ...InputValue
    }
    type {
      ...TypeRef
    }
    isDeprecated
    deprecationReason
  }
  inputFields {
    ...InputValue
  }
  interfaces {
    ...TypeRef
  }
  enumValues(includeDeprecated: true) {
    name
    description
    isDeprecated
    deprecationReason
  }
  possibleTypes {
    ...TypeRef
  }
}

fragment InputValue on __InputValue {
  name
  description
  type { ...TypeRef }
  defaultValue
}

fragment TypeRef on __Type {
  kind
  name
  ofType {
    kind
    name
    ofType {
      kind
      name
      ofType {
        kind
        name
        ofType {
          kind
          name
          ofType {
            kind
            name
          }
        }
      }
    }
  }
}
"""

class SchemaFormatError(Exception):
    pass

class NetworkError(Exception):
    pass

class ValidationError(Exception):
    pass

class GraphQLSchemaAdapter:
    def __init__(self, endpoint: str, headers: dict = None, config: dict = None):
        self.validate_endpoint(endpoint)
        self.endpoint = endpoint
        self.headers = headers if headers else {}
        self.config = config if config else {}
        self._schema_cache = None
        # <Issue: Missing validation to ensure config keys are only from the allowed set.

    def validate_endpoint(self, endpoint: str) -> None:
        parsed = urlparse(endpoint)
        if not parsed.scheme in ("http", "https") or not parsed.netloc:
            raise ValueError("Invalid GraphQL endpoint. Must be a valid URL.")

    def fetch_current_schema(self) -> dict:
        try:
            response = requests.post(
                self.endpoint,
                json={"query": INTROSPECTION_QUERY},
                headers=self.headers,
                timeout=10
            )
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            raise NetworkError(f"Network error during schema fetch: {str(e)}")

        try:
            data = response.json()
        except json.JSONDecodeError:
            raise SchemaFormatError("Invalid JSON response from server.")

        if "errors" in data:
            raise ValidationError("GraphQL introspection failed or is disabled.")

        if "__schema" not in data.get("data", {}):
            raise SchemaFormatError("No '__schema' key found in introspection response.")

        return {"__schema": data["data"]["__schema"]}

    def load_previous_schema(self, source: dict) -> dict:
        if not source or "__schema" not in source:
            raise SchemaFormatError("Previous schema does not contain '__schema'.")
        return source

    def compare_schemas(self, old_schema: dict, new_schema: dict) -> dict:
        old_types = {t["name"]: t for t in old_schema["__schema"].get("types", []) if t["name"]}
        new_types = {t["name"]: t for t in new_schema["__schema"].get("types", []) if t["name"]}

        added_types = list(set(new_types.keys()) - set(old_types.keys()))
        removed_types = list(set(old_types.keys()) - set(new_types.keys()))
        changed_fields = []

        common_types = set(old_types.keys()).intersection(new_types.keys())
        for type_name in common_types:
            old_type = old_types[type_name]
            new_type = new_types[type_name]
            if "fields" in old_type and "fields" in new_type:
                old_fields = {f["name"]: f for f in old_type["fields"] if f["name"]}
                new_fields = {f["name"]: f for f in new_type["fields"] if f["name"]}

                # Identify changed fields
                intersecting_fields = set(old_fields.keys()).intersection(new_fields.keys())
                for field_name in intersecting_fields:
                    old_repr = self.field_signature(old_fields[field_name])
                    new_repr = self.field_signature(new_fields[field_name])
                    if old_repr != new_repr:
                        changed_fields.append({
                            "type_name": type_name,
                            "field_name": field_name,
                            "old_signature": old_repr,
                            "new_signature": new_repr
                        })

        return {
            "added_types": sorted(added_types),
            "removed_types": sorted(removed_types),
            "changed_fields": changed_fields
        }

    def field_signature(self, field: dict) -> str:
        type_str = self.type_to_str(field["type"])
        args_str = []
        for arg in field.get("args", []):
            arg_type_str = self.type_to_str(arg["type"])
            args_str.append(f"{arg['name']}:{arg_type_str}")
        args_str = ",".join(sorted(args_str))
        return f"{field['name']}({args_str}):{type_str}"

    def type_to_str(self, type_obj: dict) -> str:
        # Recursively resolve the type name with nullability
        def helper(t):
            kind = t.get("kind")
            name = t.get("name")
            of_type = t.get("ofType")
            if kind == "NON_NULL":
                return f"{helper(of_type)}!"
            elif kind == "LIST":
                return f"[{helper(of_type)}]"
            else:
                return name if name else "UNKNOWN"

        return helper(type_obj)

    def update_internal_state(self, schema_diff: dict) -> None:
        # If configured to update, we replace our cache with new schema
        # or apply partial updates in a real scenario. Here, we do a simple replacement.
        if self.config.get("update_internal_state"):
            self._schema_cache = schema_diff
        # <Issue>: Cached value should be current schema, not the diff dictionary. This will lead to all cache misses and the caching mechanism will not work

    def run(self, old_schema: dict = None) -> dict:
        current_schema = self.fetch_current_schema()
        # <Issue>: Missing memory check during operations should be less than 100MB.
        if old_schema:
            old_schema_valid = self.load_previous_schema(old_schema)
            diff = self.compare_schemas(old_schema_valid, current_schema)
            if self.config.get("validate_changes"):
                # Could add extra validations here
                # <Issue>: 'validate_changes' is set but no validation report is returned or built.
                pass
            if self.config.get("print_diff"):
                pass
                 # <Issue>: 'print_diff' is set but schema differences are not printed
            self.update_internal_state(diff)
            return {
                "__schema": current_schema["__schema"],
                "added_types": diff["added_types"],
                "removed_types": diff["removed_types"],
                "changed_fields": diff["changed_fields"]
            }
        else:
            return current_schema
```